<a href="https://colab.research.google.com/github/rivkms/atp_ai/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 캐글과 코랩 연동을 위한 kaggle.jason 파일(캐글-My Account에서 다운로드) 업로드
!pip install kaggle
from google.colab import files
files.upload()

KeyboardInterrupt: ignored

In [ ]:
# kaggle.json 파일을 적절한 디렉토리로 이동
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 이 일어나지 않도록 함 
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# zoo-animal-classification 데이터셋 다운로드
!kaggle datasets download -d arashnic/earthquake-magnitude-damage-and-impact

earthquake-magnitude-damage-and-impact.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# zip파일 압축 해제 후 현재 디렉토리의 파일 리스트 보기
!unzip earthquake-magnitude-damage-and-impact.zip
!ls

Archive:  earthquake-magnitude-damage-and-impact.zip
  inflating: csv_building_damage_assessment.csv  
  inflating: csv_building_ownership_and_use.csv  
  inflating: csv_building_structure.csv  
  inflating: csv_household_demographics.csv  
  inflating: csv_household_earthquake_impact.csv  
  inflating: csv_household_resources.csv  
  inflating: csv_individual_demographics.csv  
  inflating: mapping.csv             
  inflating: ward_vdcmun_district_name_mapping.csv  
csv_building_damage_assessment.csv   csv_individual_demographics.csv
csv_building_ownership_and_use.csv   earthquake-magnitude-damage-and-impact.zip
csv_building_structure.csv	     kaggle.json
csv_household_demographics.csv	     mapping.csv
csv_household_earthquake_impact.csv  ward_vdcmun_district_name_mapping.csv
csv_household_resources.csv


In [9]:
import numpy as np
import pandas as pd

# csv 파일을 pandas dataframe으로 읽기
data = pd.read_csv('/content/csv_building_structure.csv')

# 상위 5개 행 출력
data.head()

,building_id,district_id,vdcmun_id,ward_id,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,condition_post_eq,damage_grade,technical_solution_proposed
0,120101000011,12,1207,120703,1,1,9,288,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,1,0,0,0,Damaged-Used in risk,Grade 3,Major repair
1,120101000021,12,1207,120703,1,1,15,364,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,1,0,0,0,Damaged-Repaired and used,Grade 5,Reconstruction
2,120101000031,12,1207,120703,1,1,20,384,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,Damaged-Repaired and used,Grade 2,Minor repair
3,120101000041,12,1207,120703,1,1,20,312,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,Damaged-Repaired and used,Grade 2,Minor repair
4,120101000051,12,1207,120703,1,1,30,308,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,Damaged-Repaired and used,Grade 1,Minor repair


In [10]:
del1 = ['building_id', 'district_id', 'vdcmun_id', 'ward_id', 'count_floors_pre_eq', 'count_floors_post_eq', 'height_ft_post_eq']
del2 = ['roof_type', 'plan_configuration', 'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag']
del3 = ['has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick', 'has_superstructure_timber', 'has_superstructure_bamboo']
del4 = ['has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered', 'has_superstructure_other', 'condition_post_eq', 'technical_solution_proposed']

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 762106 entries, 0 to 762105
Data columns (total 31 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   building_id                             762106 non-null  int64 
 1   district_id                             762106 non-null  int64 
 2   vdcmun_id                               762106 non-null  int64 
 3   ward_id                                 762106 non-null  int64 
 4   count_floors_pre_eq                     762106 non-null  int64 
 5   count_floors_post_eq                    762106 non-null  int64 
 6   age_building                            762106 non-null  int64 
 7   plinth_area_sq_ft                       762106 non-null  int64 
 8   height_ft_pre_eq                        762106 non-null  int64 
 9   height_ft_post_eq                       762106 non-null  int64 
 10  land_surface_condition                  762106 non-null 

In [ ]:
data.drop()

In [ ]:
data["damage_grade"].mean()

In [12]:
data.isna

<bound method DataFrame.isna of          building_id  district_id  ...  damage_grade  technical_solution_proposed
0       120101000011           12  ...       Grade 3                 Major repair
1       120101000021           12  ...       Grade 5               Reconstruction
2       120101000031           12  ...       Grade 2                 Minor repair
3       120101000041           12  ...       Grade 2                 Minor repair
4       120101000051           12  ...       Grade 1                 Minor repair
...              ...          ...  ...           ...                          ...
762101  366709001241           36  ...       Grade 5               Reconstruction
762102  366709001251           36  ...       Grade 5               Reconstruction
762103  366709001261           36  ...       Grade 5               Reconstruction
762104  366709001271           36  ...       Grade 5               Reconstruction
762105  366709001281           36  ...       Grade 5              

NameError: ignored

In [13]:
for n in del1:
  data.drop(n, axis=1, inplace = True)
for n in del2:
  data.drop(n, axis=1, inplace = True)
for n in del3:
  data.drop(n, axis=1, inplace = True)
for n in del4:
  data.drop(n, axis=1, inplace = True)
data.head()

,age_building,plinth_area_sq_ft,height_ft_pre_eq,land_surface_condition,foundation_type,ground_floor_type,other_floor_type,position,damage_grade
0,9,288,9,Flat,Other,Mud,Not applicable,Not attached,Grade 3
1,15,364,9,Flat,Other,Mud,Not applicable,Not attached,Grade 5
2,20,384,9,Flat,Other,Mud,Not applicable,Not attached,Grade 2
3,20,312,9,Flat,Other,Mud,Not applicable,Not attached,Grade 2
4,30,308,9,Flat,Other,Mud,Not applicable,Not attached,Grade 1


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 762106 entries, 0 to 762105
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   age_building            762106 non-null  int64 
 1   plinth_area_sq_ft       762106 non-null  int64 
 2   height_ft_pre_eq        762106 non-null  int64 
 3   land_surface_condition  762106 non-null  object
 4   foundation_type         762106 non-null  object
 5   ground_floor_type       762106 non-null  object
 6   other_floor_type        762106 non-null  object
 7   position                762105 non-null  object
 8   damage_grade            762094 non-null  object
dtypes: int64(3), object(6)
memory usage: 52.3+ MB


In [23]:
data.isnull().sum()


age_building               0
plinth_area_sq_ft          0
height_ft_pre_eq           0
land_surface_condition     0
foundation_type            0
ground_floor_type          0
other_floor_type           0
position                   1
damage_grade              12
dtype: int64

In [25]:
data1 = data.dropna()
data1.isnull().sum()

age_building              0
plinth_area_sq_ft         0
height_ft_pre_eq          0
land_surface_condition    0
foundation_type           0
ground_floor_type         0
other_floor_type          0
position                  0
damage_grade              0
dtype: int64